In [1]:
import nflreadpy as nfl
import pandas as pd

rosters = nfl.load_rosters().to_pandas()
qbs = rosters[rosters['position'] == 'QB']
active_qb_ids = qbs['pfr_id'].dropna().tolist()

In [2]:
active_qb_ids[0]

'RivePh00'

In [81]:
full_rosters = nfl.load_rosters(range(2010, 2026)).to_pandas()
qbs_since_2010 = full_rosters[full_rosters["position"] == "QB"]

qb_roster_unique = (
    qbs_since_2010
    .groupby("gsis_id", as_index=False)
    .agg({"pfr_id": "first",
    "full_name": "first",
    "position": "first"})
)

all_stats = nfl.load_player_stats(range(2010, 2025), "reg").to_pandas()

qb_stats = all_stats.merge(
    qb_roster_unique[["pfr_id", "gsis_id"]],
    left_on="player_id",
    right_on="gsis_id",
    how="inner"
)

qb_totals = (
    qb_stats
    .groupby(["player_id", "pfr_id"], as_index=False)["passing_yards"]
    .sum()
)

valid_qbs = qb_totals[qb_totals["passing_yards"] >= 2000]

valid_players = valid_qbs["pfr_id"].dropna().unique().tolist()

In [82]:
final_qbs = valid_qbs.merge(qb_roster_unique, left_on="pfr_id", right_on="pfr_id", how="left").drop(columns=["player_id", "gsis_id", "passing_yards"])

In [102]:
len(valid_players)

122

In [104]:
final_qbs[final_qbs["full_name"] == "Philip Rivers"]

,pfr_id,full_name,position
11,RivePh00,Philip Rivers,QB


In [121]:
wrs_since_2010 = full_rosters[full_rosters["position"] == "WR"]

wr_roster_unique = (
    wrs_since_2010
    .groupby("gsis_id", as_index=False)
    .agg({"pfr_id": "first",
    "full_name": "first",
    "position": "first"})
)

wr_stats = all_stats.merge(
    wr_roster_unique[["pfr_id", "gsis_id"]],
    left_on="player_id",
    right_on="gsis_id",
    how="inner"
)

wr_totals = (
    wr_stats
    .groupby(["player_id", "pfr_id"], as_index=False)["receiving_yards"]
    .sum()
)

valid_wr = wr_totals[wr_totals["receiving_yards"] >= 1500]

valid_wrs = valid_wr["pfr_id"].dropna().unique().tolist()

In [136]:
final_wrs = valid_wr.merge(wr_roster_unique, left_on="pfr_id", right_on="pfr_id", how="left").drop(columns=["player_id", "gsis_id", "receiving_yards"])

In [3]:
import requests
from bs4 import BeautifulSoup
import time

In [72]:
from io import StringIO

BASE_URL = "https://www.pro-football-reference.com/players"

def scrape_qb(pfr_slug):
    first_letter = pfr_slug[0]
    url = f"{BASE_URL}/{first_letter}/{pfr_slug}.htm"

    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    table = soup.find("table", {"id": "passing"})
    if table is None:
        return None

    df = pd.read_html(StringIO(str(table)))[0]

    cleaned = df[
        df["Season"].astype(str).str.match(r"^\d{4}$", na=False)
    ].copy()

    wanted_cols = [
        "Season", "Age", "Team", "Pos", "G", "GS", "Cmp", "Att", "Cmp%",
        "Yds", "TD", "Int", "Y/A", "Rate", "QBR", "AV", "Awards"
    ]

    for col in wanted_cols:
        if col not in cleaned.columns:
            cleaned.loc[:, col] = None

    final = cleaned[wanted_cols].copy()

    numeric_cols = final.columns.difference(["Team", "Pos", "Awards"])
    final.loc[:, numeric_cols] = final[numeric_cols].apply(
        pd.to_numeric, errors="coerce"
    )

    return final

In [132]:
from io import StringIO

BASE_URL = "https://www.pro-football-reference.com/players"

def scrape_wr(pfr_slug):
    first_letter = pfr_slug[0]
    url = f"{BASE_URL}/{first_letter}/{pfr_slug}.htm"

    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    table = soup.find("table", {"id": "receiving_and_rushing"})
    if table is None:
        return None

    df = pd.read_html(StringIO(str(table)), header=1)[0]

    cleaned = df[
        df["Season"].astype(str).str.match(r"^\d{4}$", na=False)
    ].copy()

    wanted_cols = [
        "Season", "Age", "Team", "Pos", "G", "Tgt", "Rec", "Yds",
        "Y/R", "TD", "Awards"
    ]

    for col in wanted_cols:
        if col not in cleaned.columns:
            cleaned.loc[:, col] = None

    final = cleaned[wanted_cols].copy()

    numeric_cols = final.columns.difference(["Team", "Pos", "Awards"])
    final.loc[:, numeric_cols] = final[numeric_cols].apply(
        pd.to_numeric, errors="coerce"
    )

    return final

In [133]:
scrape_wr(valid_wrs[0])

,Season,Age,Team,Pos,G,Tgt,Rec,Yds,Y/R,TD,Awards
0,2001,22,CAR,WR,15.0,20.0,10.0,154.0,15.4,0.0,"PB,AP-1"
1,2002,23,CAR,WR,15.0,97.0,54.0,872.0,16.1,3.0,NaN
2,2003,24,CAR,WR,16.0,141.0,88.0,1110.0,12.6,7.0,NaN
3,2004,25,CAR,WR,1.0,9.0,6.0,60.0,10.0,0.0,NaN
4,2005,26,CAR,WR,16.0,150.0,103.0,1563.0,15.2,12.0,"PB,AP-1,AP CPoY-1"
5,2006,27,CAR,WR,14.0,139.0,83.0,1166.0,14.0,8.0,PB
6,2007,28,CAR,WR,15.0,148.0,87.0,1002.0,11.5,7.0,NaN
7,2008,29,CAR,WR,14.0,129.0,78.0,1421.0,18.2,6.0,"PB,AP-2,AP OPoY-6"
8,2009,30,CAR,WR,15.0,129.0,65.0,982.0,15.1,7.0,NaN
9,2010,31,CAR,WR,14.0,100.0,46.0,554.0,12.0,2.0,NaN


In [91]:
import sqlite3

In [139]:
# Use local path for development (change to "/data/football_wordle.db" for Docker/production)
conn = sqlite3.connect("football_wordle.db")
cur = conn.cursor()

In [96]:
cur.execute("""
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pfr_id TEXT UNIQUE,
    name TEXT,
    position TEXT
)
""")

In [97]:
for _, row in final_qbs.iterrows():
    cur.execute(
        """
        INSERT OR IGNORE INTO players (pfr_id, name, position)
        VALUES (?, ?, ?)
        """,
        (row["pfr_id"], row["full_name"], row["position"])
    )

conn.commit()

In [100]:
cur.execute("""
CREATE TABLE IF NOT EXISTS passing_seasons (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER,
    season INTEGER,
    age INTEGER,
    team TEXT,
    games INTEGER,
    games_started INTEGER,
    completions INTEGER,
    attempts INTEGER,
    completion_pct REAL,
    yards INTEGER,
    touchdowns INTEGER,
    interceptions INTEGER,
    yards_per_attempt REAL,
    passer_rating REAL,
    qbr REAL,
    av REAL,
    awards TEXT,
    FOREIGN KEY(player_id) REFERENCES players(id)
)
""")

In [105]:
for pfr_id in valid_players:
    seasons = scrape_qb(pfr_id)
    if seasons is None or seasons.empty:
        continue

    cur.execute(
        "SELECT id FROM players WHERE pfr_id = ?",
        (pfr_id,)
    )
    player_id = cur.fetchone()[0]

    rows = []
    for _, s in seasons.iterrows():
        rows.append((
            player_id,
            s["Season"], s["Age"], s["Team"], s["G"], s["GS"],
            s["Cmp"], s["Att"], s["Cmp%"], s["Yds"],
            s["TD"], s["Int"], s["Y/A"],
            s["Rate"], s["QBR"], s["AV"], s["Awards"]
        ))

    cur.executemany(
        """
        INSERT INTO passing_seasons (
            player_id, season, age, team, games, games_started,
            completions, attempts, completion_pct, yards,
            touchdowns, interceptions, yards_per_attempt,
            passer_rating, qbr, av, awards
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        rows
    )

    conn.commit()
    print(f"Inserted {len(rows)} seasons for {pfr_id}")
    time.sleep(7)


Inserted 17 seasons for HassMa00
Inserted 18 seasons for MannPe00
Inserted 23 seasons for BradTo00
Inserted 20 seasons for BreeDr00
Inserted 17 seasons for McCoJo01
Inserted 9 seasons for GarrDa00
Inserted 14 seasons for PalmCa00
Inserted 9 seasons for GrosRe00
Inserted 16 seasons for SchaMa00
Inserted 16 seasons for MannEl00
Inserted 18 seasons for RoetBe00
Inserted 22 seasons for RivePh00
Inserted 16 seasons for SmitAl03
Inserted 21 seasons for RodgAa00
Inserted 9 seasons for CampJa00
Inserted 11 seasons for OrtoKy00
Inserted 13 seasons for AndeDe00
Inserted 16 seasons for CassMa00
Inserted 17 seasons for FitzRy00
Inserted 6 seasons for YounVi00
Inserted 12 seasons for CutlJa00
Inserted 12 seasons for ClemKe00
Inserted 9 seasons for JackTa00
Inserted 7 seasons for WhitCh02
Inserted 6 seasons for KolbKe00
Inserted 7 seasons for StanDr00
Inserted 15 seasons for RyanMa00
Inserted 20 seasons for FlacJo00
Inserted 13 seasons for HennCh01
Inserted 9 seasons for FlynMa00
Inserted 17 seasons

In [ ]:
def get_random_qb_career():
    conn = sqlite3.connect("football_wordle.db")

    player = pd.read_sql("""
        SELECT DISTINCT p.id, p.name
        FROM players p
        JOIN passing_seasons ps
        ON p.id = ps.player_id
        ORDER BY RANDOM()
        LIMIT 1;
    """, conn).iloc[0]

    seasons = pd.read_sql("""
        SELECT *
        FROM passing_seasons
        WHERE player_id = ?
        ORDER BY season
    """, conn, params=(int(player["id"]),))

    conn.close()
    return player['name'], seasons


In [107]:
name, df = get_random_qb_career()
name

'Sam Howell'

In [137]:
final_wrs

,pfr_id,full_name,position
0,SmitSt01,Steve Smith,WR
1,MossSa00,Santana Moss,WR
2,WaynRe00,Reggie Wayne,WR
3,GaffJa00,Jabar Gaffney,WR
4,BranDe00,Deion Branch,WR
...,...,...,...
211,AddiJo00,Jordan Addison,WR
212,DownJo00,Josh Downs,WR
213,FlowZa00,Zay Flowers,WR
214,NacuPu00,Puka Nacua,WR


In [140]:
# Insert WRs into players table
for _, row in final_wrs.iterrows():
    cur.execute(
        """
        INSERT OR IGNORE INTO players (pfr_id, name, position)
        VALUES (?, ?, ?)
        """,
        (row["pfr_id"], row["full_name"], row["position"])
    )

conn.commit()
print(f"Inserted {len(final_wrs)} WRs into players table")


Inserted 216 WRs into players table


In [141]:
# Create receiving_seasons table
cur.execute("""
CREATE TABLE IF NOT EXISTS receiving_seasons (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER,
    season INTEGER,
    age INTEGER,
    team TEXT,
    games INTEGER,
    targets INTEGER,
    receptions INTEGER,
    yards INTEGER,
    yards_per_reception REAL,
    touchdowns INTEGER,
    awards TEXT,
    FOREIGN KEY(player_id) REFERENCES players(id)
)
""")
conn.commit()
print("Created receiving_seasons table")


Created receiving_seasons table


In [142]:
import tqdm


In [144]:
# Populate WR data (this will take a while due to rate limiting)
for pfr_id in tqdm.tqdm(valid_wrs):
    # Check if already exists
    cur.execute(
        "SELECT id FROM players WHERE pfr_id = ?",
        (pfr_id,)
    )
    result = cur.fetchone()
    if result is None:
        print(f"Skipping {pfr_id} - not in players table")
        continue
    
    player_id = result[0]
    
    # Check if seasons already exist for this player
    cur.execute(
        "SELECT COUNT(*) FROM receiving_seasons WHERE player_id = ?",
        (player_id,)
    )
    if cur.fetchone()[0] > 0:
        print(f"Skipping {pfr_id} - already has receiving data")
        continue
    
    # Scrape WR data
    seasons = scrape_wr(pfr_id)
    if seasons is None or seasons.empty:
        print(f"No data for {pfr_id}")
        continue

    rows = []
    for _, s in seasons.iterrows():
        rows.append((
            player_id,
            s["Season"], s["Age"], s["Team"], s["G"],
            s["Tgt"], s["Rec"], s["Yds"],
            s["Y/R"], s["TD"], s["Awards"]
        ))

    cur.executemany(
        """
        INSERT INTO receiving_seasons (
            player_id, season, age, team, games,
            targets, receptions, yards,
            yards_per_reception, touchdowns, awards
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        rows
    )

    conn.commit()
    #print(f"Inserted {len(rows)} seasons for {pfr_id}")
    time.sleep(7)  # Rate limiting

print("Done populating WR data!")

 24%|██▍       | 52/216 [06:17<14:14,  5.21s/it]

No data for McClDe00


 44%|████▍     | 95/216 [11:26<10:33,  5.24s/it]

No data for PattCo00


100%|██████████| 216/216 [53:13<00:00, 14.79s/it]   

Done populating WR data!


In [145]:
# Extract RB data from rosters
rbs_since_2010 = full_rosters[full_rosters["position"] == "RB"]

rb_roster_unique = (
    rbs_since_2010
    .groupby("gsis_id", as_index=False)
    .agg({"pfr_id": "first",
    "full_name": "first",
    "position": "first"})
)

rb_stats = all_stats.merge(
    rb_roster_unique[["pfr_id", "gsis_id"]],
    left_on="player_id",
    right_on="gsis_id",
    how="inner"
)

rb_totals = (
    rb_stats
    .groupby(["player_id", "pfr_id"], as_index=False)["rushing_yards"]
    .sum()
)

# Filter to RBs with at least 1500 rushing yards
valid_rb = rb_totals[rb_totals["rushing_yards"] >= 1500]
valid_rbs = valid_rb["pfr_id"].dropna().unique().tolist()

print(f"Found {len(valid_rbs)} RBs with 1500+ rushing yards")


Found 141 RBs with 1500+ rushing yards


In [147]:
# Create final_rbs DataFrame
final_rbs = valid_rb.merge(
    rb_roster_unique, 
    left_on="pfr_id", 
    right_on="pfr_id", 
    how="left"
).drop(columns=["player_id", "gsis_id", "rushing_yards"])

print(f"Total RBs to add: {len(final_rbs)}")
final_rbs.head()


Total RBs to add: 141


,pfr_id,full_name,position
0,McGaWi00,Willis McGahee,RB
1,JackSt00,Steven Jackson,RB
2,TurnMi00,Michael Turner,RB
3,BensCe00,Cedric Benson,RB
4,GoreFr00,Frank Gore,RB


In [148]:
# Insert RBs into players table
for _, row in final_rbs.iterrows():
    cur.execute(
        """
        INSERT OR IGNORE INTO players (pfr_id, name, position)
        VALUES (?, ?, ?)
        """,
        (row["pfr_id"], row["full_name"], row["position"])
    )

conn.commit()
print(f"Inserted {len(final_rbs)} RBs into players table")


Inserted 141 RBs into players table


In [150]:
# Scrape function for RBs
def scrape_rb(pfr_slug):
    first_letter = pfr_slug[0]
    url = f"{BASE_URL}/{first_letter}/{pfr_slug}.htm"

    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # RBs use the "rushing_and_receiving" table on PFR
    table = soup.find("table", {"id": "rushing_and_receiving"})
    if table is None:
        return None

    df = pd.read_html(StringIO(str(table)), header=1)[0]

    cleaned = df[
        df["Season"].astype(str).str.match(r"^\d{4}$", na=False)
    ].copy()

    # Columns we want for RBs
    wanted_cols = [
        "Season", "Age", "Team", "Pos", "G", 
        "Att", "Yds", "Y/A", "TD",  # Rushing stats
        "Rec", "Yds.1",  # Receiving stats (Yds.1 is receiving yards)
        "Awards"
    ]

    for col in wanted_cols:
        if col not in cleaned.columns:
            cleaned.loc[:, col] = None

    final = cleaned[wanted_cols].copy()

    # Rename Yds.1 to Rec_Yds for clarity
    final = final.rename(columns={"Yds.1": "Rec_Yds"})

    numeric_cols = final.columns.difference(["Team", "Pos", "Awards"])
    final.loc[:, numeric_cols] = final[numeric_cols].apply(
        pd.to_numeric, errors="coerce"
    )

    return final

# Test the function
print("Testing scrape_rb...")
test_rb = scrape_rb(valid_rbs[0])
if test_rb is not None:
    print(f"Successfully scraped {valid_rbs[0]}")
    display(test_rb.head())
else:
    print(f"Failed to scrape {valid_rbs[0]}")


Testing scrape_rb...
Successfully scraped McGaWi00


,Season,Age,Team,Pos,G,Att,Yds,Y/A,TD,Rec,Rec_Yds,Awards
0,2004,23,BUF,RB,16.0,284.0,1128.0,4.0,13.0,22.0,169.0,AP CPoY-3
1,2005,24,BUF,RB,16.0,325.0,1247.0,3.8,5.0,28.0,178.0,NaN
2,2006,25,BUF,RB,14.0,259.0,990.0,3.8,6.0,18.0,156.0,NaN
3,2007,26,BAL,RB,15.0,294.0,1207.0,4.1,7.0,43.0,231.0,PB
4,2008,27,BAL,RB,13.0,170.0,671.0,3.9,7.0,24.0,173.0,NaN


In [151]:
# Create rushing_seasons table
cur.execute("""
CREATE TABLE IF NOT EXISTS rushing_seasons (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER,
    season INTEGER,
    age INTEGER,
    team TEXT,
    games INTEGER,
    attempts INTEGER,
    yards INTEGER,
    yards_per_attempt REAL,
    touchdowns INTEGER,
    receptions INTEGER,
    receiving_yards INTEGER,
    awards TEXT,
    FOREIGN KEY(player_id) REFERENCES players(id)
)
""")
conn.commit()
print("Created rushing_seasons table")


Created rushing_seasons table


In [155]:
# Populate RB data (this will take a while due to rate limiting)
for pfr_id in tqdm.tqdm(valid_rbs):
    # Check if already exists
    cur.execute(
        "SELECT id FROM players WHERE pfr_id = ?",
        (pfr_id,)
    )
    result = cur.fetchone()
    if result is None:
        print(f"Skipping {pfr_id} - not in players table")
        continue
    
    player_id = result[0]
    
    # Check if seasons already exist for this player
    cur.execute(
        "SELECT COUNT(*) FROM rushing_seasons WHERE player_id = ?",
        (player_id,)
    )
    if cur.fetchone()[0] > 0:
        continue  # Already has data
    
    # Scrape RB data
    seasons = scrape_rb(pfr_id)
    if seasons is None or seasons.empty:
        print(f"No data for {pfr_id}")
        continue

    rows = []
    for _, s in seasons.iterrows():
        rows.append((
            player_id,
            s["Season"], s["Age"], s["Team"], s["G"],
            s["Att"], s["Yds"], s["Y/A"], s["TD"],
            s["Rec"], s["Rec_Yds"], s["Awards"]
        ))

    cur.executemany(
        """
        INSERT INTO rushing_seasons (
            player_id, season, age, team, games,
            attempts, yards, yards_per_attempt, touchdowns,
            receptions, receiving_yards, awards
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        rows
    )

    conn.commit()
    time.sleep(7)  # Rate limiting

print("Done populating RB data!")

100%|██████████| 141/141 [17:05<00:00,  7.27s/it]

Done populating RB data!
